In [18]:
import psycopg2
import boto3
import configparser
import pandas as pd 
import numpy as np 
from io import StringIO

In [4]:
Config = configparser.ConfigParser()
Config.read_file(open('/Users/milesklingenberg/Documents/Personal/AWS/AWS_Keys'))
KEY = Config.get('AWS', 'AWS_ACCESS_KEY_ID')
SECRET = Config.get('AWS', 'Secret')
s3client = boto3.client('s3', 
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)
file = s3client.get_object(Bucket = 'problem1', Key = 'prob1.csv')
prob_1 = pd.read_csv(file['Body'])
##Putting it into my own s3 bucket. 

In [5]:
prob_1

,index,first_float,first_varchar,second_float,first_number,second_varchar
0,0,8.53,tintype,0.460945,260784,countermeasure
1,1,5.52,For,0.822112,310733,malleable
2,2,1.33,You,0.627088,434806,dinnerware
3,3,9.97,percept,0.105301,776341,schema
4,4,3.44,Allentown,0.232980,442039,Elisha
...,...,...,...,...,...,...
2474995,2474995,9.45,Canis,0.394791,155516,trompe
2474996,2474996,1.59,You,0.847691,513747,postman
2474997,2474997,8.32,Hines,0.945701,287178,Kalmuk
2474998,2474998,4.67,divest,0.099642,70585,carve


In [8]:
def create_database_tables(): 
    """
    connecting to my local db (can see my password, but it's local server only)
    """ 
    
    conn = psycopg2.connect("host=localhost user=postgres password=Hockey3131!  port = 5432")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
  
    return cur, conn 

def drop_table(cur, conn): 
    cur.execute("DROP TABLE IF EXISTS problem_1")
    conn.commit() 
    
def create_tables(cur, conn): 
    cur.execute("""CREATE TABLE IF NOT EXISTS problem_1 
    (index int, 
    first_float float, 
    first_varchar text, 
    second_float float, 
    first_number int, 
    second_varchar text)""")

def process_data(cur, conn): 
    
       sio = StringIO() 
       sio.write(prob_1.to_csv(index=None, header = None))
       sio.seek(0) 
        
       with conn.cursor() as c: 
            c.copy_from(sio, "problem_1", columns=prob_1.columns, sep = ',')
            conn.commit()
                    
def main(): 
    cur, conn = create_database_tables() 
    
    drop_table(cur, conn) 
    create_tables(cur, conn)
    process_data(cur, conn)
    
    conn.close() 
if __name__ == "__main__": 
    main()

In [ ]:
--Problem 1 SQL queries 

SELECT * 
FROM public.problem_1  
WHERE index = 310733 OR index = 1

--Testing to see if the is a hidden number based on the sequence of the ints or floats 

---###Second Query 

WITH temp_1 AS (SELECT *, 
(first_float/second_float)  as third_float 
FROM public.problem_1
Fetch first 10 rows only) 

--I think there is a hidden essage if I can figure out the seqeunce of operations for these float and ints 
--suspect it will lead me to a 

SELECT i.*, 
(i.first_number/third_float) as fourth_float 
FROM temp_1 i 

----##Third Query 

WITH temp_1 AS (SELECT 
*, 
(first_float*first_number) as test_float
FROM public.problem_1
Fetch first 10 rows only) 

SELECT *, 
(test_float/first_float) as test_2_float 
FROm temp1  

---okay so those are based on each other 
---probably not the answer though 

---###Query 4, could it have to do with the rank? 

SELECT 
    *,
    RANK () OVER ( 
            ORDER BY first_float, second_float, first_number
    ) rank_number 
FROM 
    problem_1 


--### Intersting, can't find what the key is 

SELECT *, 
CASE 
    WHEN first_float > 0 THEN ((second_float/first_float)*first_number) ELSE 0 END test
FROM problem_1  

##Can't figure out what the cipher is, I see that the texts columns create a string based on the num/int columns, just
##not positive what the logic is. 

In [ ]:
SELECT COUNT (DISTINCT i.street_address) 
FROM problem_2_2 i 
-- in the second xlsx doc there are 2203
--problem_2_2 is the table containing the second link in question 2 

SELECT COUNT (DISTINCT "addressLineOne")
--Not sure what is happening here 
FROM problem_2_1_1 i 

--The first table has 312 distinct values 
-- problem_2_1_1 is the first link in question 2 

--Now we can check for whether any in the first group/table exist in the second 

WITH flag_table AS (SELECT "addressLineOne", 
1 AS FLAG_column
FROM problem_2_1_1 
WHERE "addressLineOne" IN (SELECT DISTINCT k.street_address FROM problem_2_2 k))

---There are 46 records that ecist in both. 
---The thing you could do is create a temp table, if you need to flag records from either table, as above; 
---Then you can just join to either table 

SELECT i.*, 
CASE 
        WHEN k.flag_column = 1 THEN 1 ELSE 0 END flag_c
FROM problem_2_1_1 i 
LEFT JOIN flag_table k ON i."addressLineOne" = k."addressLineOne"

--You could do the same for the other table if needed.DELETE FROM
--the 3 part of question 2 calls for the Headstart data to be referenced with a flag. 
--As mentioned problem_2_1_1 is that data set. 

--Given more time I could make an ETL process daily/ or when these tables receive new data that does not ingest duplicates. 
--Avoiding case statements is always ideal. 
##Note that typically I wouldn't use double quotes but was having trouble with the xlsx code, etc. If I need
##to make available the code for sql insert I can. 

In [39]:
##Question 3 
##For question three I will be doing problem 2 because I haven't yet figured out the cipher to question 1. 
conn = psycopg2.connect("host=localhost user=postgres password=Hockey3131!  port = 5432")
##Again using the same conection 
cur = conn.cursor() 
result = cur.execute("""SELECT * FROM problem_2_1_1""")
result = cur.fetchall()
#Only using sql for pulling in. 


In [34]:
from pandas import DataFrame 

In [56]:
df_two_one = DataFrame(query_results, columns = ['name', 'typestring', 'street_address', 'addresslinetwo', 'city', 'state', 'zipfive', 'zipfour', 'county', 'ispoboxlocation', 'phone', 'drivingdirection', 'grantnumber', 'delegatenumber', 'programname', 'programaddresslineone', 'programmaddresslinetwo', 'programcity', 'programstate', 'programszipfive', 'proramzipfour', 'programcounty', 'programphone', 'programregistrationphone', 'latitude', 'longitude_'])

In [57]:
df_two_one.head(2)

,name,typestring,street_address,addresslinetwo,city,state,zipfive,zipfour,county,ispoboxlocation,...,programmaddresslinetwo,programcity,programstate,programszipfive,proramzipfour,programcounty,programphone,programregistrationphone,latitude,longitude_
0,Peshastin Center,Head Start,10375 Mill Rd,,Peshastin,WA,98847,9733,Chelan County,false,...,,Wenatchee,WA,98801,-1,Chelan County,(509) 548-7614,(509)662-0683 - 222,47.5745,-120.6097
1,West Side Early Learning Center,Early Head Start,1510 9th Street,,Wenatchee,WA,98801,1655,Chelan County,false,...,,Wenatchee,WA,98801,-1,Chelan County,(509) 663-5179,(509)662-0683 - 222,47.4339,-120.3401


In [41]:
results_2 = cur.execute("""SELECT * FROM problem_2_2""")
results_2 = cur.fetchall()
df_two_two = DataFrame(results_2, columns = ['school_id', 'stat_school_id', 'district_id', 'state_id', 'low_grade', 'high_grade', 'school_name', 'district', 'county_name', 'street_address', 'city', 'state', 'zip', 'zip4dig', 'phone', 'locale_code', 'locale',  'chart', 'maagnet', 'title_i_school', 'title_1_school_wide', 'student', 'teacher', 'stud_teach', 'free_lunch', 'reduced_lunch'])

In [42]:
df_two_two.head(2)

,school_id,stat_school_id,district_id,state_id,low_grade,high_grade,school_name,district,county_name,street_address,...,locale,chart,maagnet,title_i_school,title_1_school_wide,student,teacher,stud_teach,free_lunch,reduced_lunch
0,530486002475,WA-31025-1656,5304860,WA-31025,06,08,10th Street School,Marysville School District,Snohomish County,7204 27th Ave NE,...,Suburb: Midsize,No,No,–,–,174.00000,7.00000,24.9000000,34.00000,8.00000
1,530270001270,WA-06114-1646,5302700,WA-06114,KG,12,49th Street Academy,Evergreen School District (Clark),Clark County,14619B NE 49th Street,...,City: Midsize,No,No,Yes,Yes,97.00000,12.14000,8.0000000,58.00000,10.00000


In [46]:
##Will work through 1-3 for each 
df_two_one['addresslineone'].nunique()

312

In [48]:
##First has 312, same as SQL. 
df_two_two['street_address'].nunique() 

2203

In [62]:
##And the second has same as above as well (for sql)
##You mention you use dataframes I will do the same 
flag_check = pd.merge(df_two_one, df_two_two, how = 'inner', on=['street_address'])

In [63]:
len(flag_check)

47

In [67]:
##Good, so we got the same number. I renamed the address line one for the purposes 
##of this pipeline, you could alias it also. 

flag_check['flag'] = 1 
##Only care about the two columns, and then we will bring back to headstart 
flag_check[['street_address', 'flag']]

#headstart
final_table = pd.merge(df_two_one, flag_check, how = 'left', on=['street_address']) 

In [70]:
final_table.tail(10)
##if we wanted we could give the boolean 0 for that flag column but will move on 
##at this point. 

,name_x,typestring_x,street_address,addresslinetwo_x,city,state,zipfive_x,zipfour_x,county_x,ispoboxlocation_x,...,chart,maagnet,title_i_school,title_1_school_wide,student,teacher,stud_teach,free_lunch,reduced_lunch,flag
350,Hough Early Learning Center,Early Head Start,1801 Esther St,,Vancouver,WA,98660,2558,Clark County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,EWU Early Head Start Socialization/Meeting Fac...,Early Head Start,1600 W 1st St,,Newport,WA,99156,9008,Pend Oreille County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,Newport,Head Start,200 S Calispel Ave,,Newport,WA,99156,9718,Pend Oreille County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,Quileute Head Start,American Indian and Alaska Native,8 By Yak Loop,,La Push,WA,98350,-1,Clallam County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,Greyhound Way Center - Head Start / ECEAP,Head Start and Early Head Start,530 NW Greyhound Way,,Pullman,WA,99163,4235,Whitman County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,LONG BEACH HEAD START,Head Start,2611 Pacific Ave N,Pacific County,Long Beach,WA,98631,3864,Pacific County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,MMC Makah Head Start,American Indian and Alaska Native,80 Ba'adah Village Loop Rd Box 115,,Neah Bay,WA,98357,-1,Clallam County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,Fair St. Early Head Start Center,Early Head Start,1483 Fair St,,Clarkston,WA,99403,2359,Asotin County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,Chestnut St. Early Head Start,Early Head Start,1260 Chestnut St,,Clarkston,WA,99403,2557,Asotin County,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,Parkway HS,Head Start,1103 4th St,,Clarkston,WA,99403,2603,Asotin County,false,...,No,No,Yes,Yes,322.00000,21.00000,15.3000000,173.00000,41.00000,1.0
